In [1]:
import os
import tensorflow as tf
import mlflow
import mlflow.keras
from sklearn.metrics import precision_score, recall_score, accuracy_score
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [3]:
#image augmentation
train_datagen = ImageDataGenerator(
          rescale=1./255,
          shear_range=0.2,
          zoom_range=0.2,
          horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
          'E:/Deep Learning/TENSORFLOW/rice_image_detection/output_dataset/train',
          target_size=(64, 64),
          batch_size=32)

Found 60000 images belonging to 5 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
          'E:/Deep Learning/TENSORFLOW/rice_image_detection/output_dataset/test',
          target_size=(64, 64),
          batch_size=32)

Found 15000 images belonging to 5 classes.


In [5]:
MLFLOW_TRACKING_URI = "https://dagshub.com/karmakaragradwip02/rice_image_detection_cnn.mlflow"
os.environ['MLFLOW_TRACKING_URI'] = MLFLOW_TRACKING_URI
os.environ['MLFLOW_TRACKING_USERNAME'] = 'karmakaragradwip02'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '9ccb0f28354fcca6469017b32544fa0704b9c343'

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("CNN Classifier")

<Experiment: artifact_location='mlflow-artifacts:/3d9fa9ea225b44c187c7461de0ea5637', creation_time=1718721506518, experiment_id='0', last_update_time=1718721506518, lifecycle_stage='active', name='CNN Classifier', tags={}>

In [7]:
weight_decay = 1e-4  # Weight decay factor
learning_rate = 1e-3  # Custom learning rate

In [8]:
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3],
                           kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu',
                          kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
    tf.keras.layers.Dense(5, activation='sigmoid')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

cnn.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
with mlflow.start_run() as run:
    try:
        mlflow.log_param('weight_decay', weight_decay)
        mlflow.log_param('learning_rate', learning_rate)
        mlflow.log_param('epochs', 5)
        # Fit the model
        history = cnn.fit(x=training_set, validation_data=test_set, epochs=5)

        # Get predictions
        y_pred = np.argmax(cnn.predict(test_set), axis=1)
        y_true = np.argmax(test_set.labels, axis=1)

        # Calculate precision and recall
        precision = precision_score(y_true, y_pred, average='macro')
        recall = recall_score(y_true, y_pred, average='macro')
        accuracy = accuracy_score(y_true, y_pred)
        
        # Log metrics
        mlflow.log_metric('accuracy', accuracy)
        mlflow.log_metric('precision', precision)
        mlflow.log_metric('recall', recall)

        # Log the entire training history
        # Log the entire training history
        for epoch in range(len(history.history['accuracy'])):
            mlflow.log_metric('train_accuracy', history.history['accuracy'][epoch], step=epoch)
            mlflow.log_metric('val_accuracy', history.history['val_accuracy'][epoch], step=epoch)
            mlflow.log_metric('train_loss', history.history['loss'][epoch], step=epoch)
            mlflow.log_metric('val_loss', history.history['val_loss'][epoch], step=epoch)
        
        mlflow.pytorch.log_model(cnn, "model")
    except Exception as e:
        print(f"Exception during training: {e}")
    finally:
        mlflow.end_run()

Epoch 1/5
1875/1875 [==============================] - 745s 396ms/step - loss: 0.2534 - accuracy: 0.9113 - val_loss: 0.0818 - val_accuracy: 0.9823
Epoch 2/5
1875/1875 [==============================] - 291s 155ms/step - loss: 0.1173 - accuracy: 0.9697 - val_loss: 0.0734 - val_accuracy: 0.9855
Epoch 3/5
1875/1875 [==============================] - 297s 158ms/step - loss: 0.0943 - accuracy: 0.9782 - val_loss: 0.0501 - val_accuracy: 0.9932
Epoch 4/5
1875/1875 [==============================] - 312s 166ms/step - loss: 0.0833 - accuracy: 0.9809 - val_loss: 0.0528 - val_accuracy: 0.9921
Epoch 5/5
469/469 [==============================] - 27s 57ms/step
Exception during training: axis 1 is out of bounds for array of dimension 1
